In [30]:
import pandas as pd
import plotly.express as px

In [31]:
# Import data and split relevant details
log_path = "fan_control.log"
with open(log_path, "r") as f:
    lines = f.readlines()
temp_lines = [l for l in lines if "CPU Temperature:" in l]
high_lines = [l for l in lines if "Setting pin GPIO17 to HIGH" in l]

## How many times did the fan run per day?

In [32]:
dates = {}
for l in high_lines:
    date = pd.to_datetime(l[1:11])
    if date not in dates.keys():
        dates[date] = 1
    else:
        dates[date] += 1

In [33]:
df = pd.DataFrame.from_dict(dates, orient="index", columns=["count"])
fig = px.bar(df, x=df.index, y="count")
fig.show()

## Core temperature over time

In [34]:
temp_data = []
for l in temp_lines:
    parts = l.split("] ")
    date = parts[0][1:]
    temp = float(parts[1][-8:-3])
    temp_data.append((date, temp))
    

In [35]:
dates, temperatures = zip(*temp_data)
dates = [pd.to_datetime(date) for date in dates]
df = pd.DataFrame({"Date": dates, "Temperature": temperatures})
fig = px.line(df, x="Date", y="Temperature", labels={"Temperature": "Temperature (°C)"}, title="Temperature Over Time")
fig.show()


## Average core temperature per day

In [36]:
from ast import Tuple
from pandas import Timestamp


running_avg = lambda x, n, p: ((n - 1) / n * p) + (x / n)
avgs_per_day: dict[Timestamp, tuple[float, int]] = {}
for l in temp_lines:
    date: Timestamp = pd.to_datetime(l[1:11])
    temp = float(l.split("] ")[1][-8:-3])
    if date in avgs_per_day.keys():
        avgs_per_day[date] = (running_avg(temp, avgs_per_day[date][1],
                                          avgs_per_day[date][0]), 
                              avgs_per_day[date][1] + 1)
    else:
        avgs_per_day[date] = (temp, 1)

avgs_per_real_day: dict[Timestamp, float] = {k: v[0] for k, v in avgs_per_day.items()}

In [37]:
df = pd.DataFrame.from_dict(avgs_per_real_day, orient="index", columns=["count"])
fig = px.scatter(df, x=df.index, y="count")
fig.show()